In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
device = "cpu"
model_path = "ibm-granite/granite-4.0-h-1b"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

model.eval()

The fast path is not available because on of `(selective_state_update, causal_conv1d_fn, causal_conv1d_update)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


GraniteMoeHybridForCausalLM(
  (model): GraniteMoeHybridModel(
    (embed_tokens): Embedding(100352, 1536, padding_idx=100256)
    (layers): ModuleList(
      (0-4): 5 x GraniteMoeHybridDecoderLayer(
        (input_layernorm): GraniteMoeHybridRMSNorm((1536,), eps=1e-05)
        (post_attention_layernorm): GraniteMoeHybridRMSNorm((1536,), eps=1e-05)
        (shared_mlp): GraniteMoeHybridMLP(
          (activation): SiLU()
          (input_linear): Linear(in_features=1536, out_features=8192, bias=False)
          (output_linear): Linear(in_features=4096, out_features=1536, bias=False)
        )
        (mamba): GraniteMoeHybridMambaLayer(
          (act): SiLU()
          (conv1d): Conv1d(3328, 3328, kernel_size=(4,), stride=(1,), padding=(3,), groups=3328)
          (in_proj): Linear(in_features=1536, out_features=6448, bias=False)
          (norm): GraniteMoeHybridRMSNormGated()
          (out_proj): Linear(in_features=3072, out_features=1536, bias=False)
        )
      )
      (5): G

In [5]:
chat = [
    { 
    "role": "user", 
    "content": "Say Hello to me" 
    },
]

chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
# tokenize the text
input_tokens = tokenizer(chat, return_tensors="pt").to(device)
# generate output tokens
output = model.generate(**input_tokens, max_new_tokens=100)
# decode output tokens into text
output = tokenizer.batch_decode(output)
# print output
print(output[0])

<|start_of_role|>system<|end_of_role|>You are a helpful assistant. Please ensure responses are professional, accurate, and safe.<|end_of_text|>
<|start_of_role|>user<|end_of_role|>Say Hello to me<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>Hello! It's great to meet you. How can I assist you today?<|end_of_text|>
